In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! apt install tesseract-ocr
! apt install libtesseract-dev
! pip install Pillow
! pip install pytesseract
! sudo apt-get install tesseract-ocr-script-hang tesseract-ocr-script-hang-vert

In [ ]:
!git clone https://github.com/tesseract-ocr/tesseract

In [ ]:
!git clone https://github.com/tesseract-ocr/langdata

In [ ]:
!git clone https://github.com/tesseract-ocr/tessdata_best

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import glob
import os
import pytesseract
from PIL import ImageEnhance, ImageFilter, Image
if __name__ == "__main__":
    
    #os.chdir('/content/drive/MyDrive/ProjectStudy/workbooks/raw_data/Pyeonga/2021/2021_06_ga/')
    image='/content/drive/MyDrive/ts/swpg.jpg'
    white = [255,255,255]
    img=cv2.imread(image)
    constant= cv2.copyMakeBorder(img,200,100,100,100,cv2.BORDER_CONSTANT,value=white)

    cv2_imshow(img)
    cv2_imshow(constant)
    d=pytesseract.image_to_string(constant, lang='eng+Hangul')
    print("d:", d)


In [ ]:
import json
import time
import cv2
import requests
import sys
from google.colab.patches import cv2_imshow
LIMIT_PX = 1024
LIMIT_BYTE = 1024*1024  # 1MB
LIMIT_BOX = 40


def kakao_ocr_resize(image_path: str):
    """
    ocr detect/recognize api helper
    ocr api의 제약사항이 넘어서는 이미지는 요청 이전에 전처리가 필요.

    pixel 제약사항 초과: resize
    용량 제약사항 초과  : 다른 포맷으로 압축, 이미지 분할 등의 처리 필요. (예제에서 제공하지 않음)

    :param image_path: 이미지파일 경로
    :return:
    """
    image = cv2.imread(image_path)
 
    height, width, _ = image.shape

    if LIMIT_PX < height or LIMIT_PX < width:
        ratio = float(LIMIT_PX) / max(height, width)
        image = cv2.resize(image, None, fx=ratio, fy=ratio)
        height, width, _ = height, width, _ = image.shape

        # api 사용전에 이미지가 resize된 경우, recognize시 resize된 결과를 사용해야함.
        image_path = "{}_resized.jpg".format(image_path)
        cv2.imwrite(image_path, image)

        return image_path
    return None


def kakao_ocr(image_path: str, appkey: str):
    """
    OCR api request example
    :param image_path: 이미지파일 경로
    :param appkey: 카카오 앱 REST API 키
    """
    API_URL = 'https://dapi.kakao.com/v2/vision/text/ocr'

    headers = {'Authorization': 'KakaoAK {}'.format(appkey)}

    image = cv2.imread(image_path)
    jpeg_image = cv2.imencode(".jpg", image)[1]
    data = jpeg_image.tobytes()


    return requests.post(API_URL, headers=headers, files={"image": data})


def main():
    if len(sys.argv) != 3:
        print("Please run with args: $ python example.py /path/to/image appkey")
    image_path, appkey = '/content/drive/MyDrive/ts/swpg.png', '앱키'
    white = [255,255,255]
    img=cv2.imread(image_path)
    constant= cv2.copyMakeBorder(img,200,100,100,100,cv2.BORDER_CONSTANT,value=white)
    image_path='/content/drive/MyDrive/ts/swpg.jpg'
    cv2.imwrite(image_path, constant)
    cv2_imshow(constant)
    time.sleep(2)
    resize_impath = kakao_ocr_resize(image_path)
    if resize_impath is not None:
        image_path = resize_impath
        print("원본 대신 리사이즈된 이미지를 사용합니다.")

    output = kakao_ocr(image_path, appkey).json()
    print("[OCR] output:\n{}\n".format(json.dumps(output, sort_keys=True,ensure_ascii=False, indent=2)))


if __name__ == "__main__":
    main()
  



In [ ]:
!pip install --upgrade imutils
!pip install pyimagesearch

In [ ]:
from pyimagesearch.transform import four_point_transform
from skimage.filters import threshold_local
import numpy as np
import argparse
import cv2
import imutils
def scanphoto(imgfile):
    image=cv2.imread(imgfile)
    ratio= image.shape[0]/500.0
    orig=image.copy()
    image=imutils.resize(image,height=500)

    gray=cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray=cv2.GaussianBlur(gray,(5,5),0)
    edged=cv2.Canny(gray,75,200)

    print("STEP1")
    cv2_imshow(image)
    cv2_imshow(edged)


In [ ]:
def readPage(name,img):
    #path='/content/drive/MyDrive/GRADING_Study/kh/header/'
    cv2_imshow(img)
    #print(path+name)
    #cv2.imwrite(path+name, img)

In [ ]:
def cropTop(image):
    src=cv2.imread(image)
    #cv2_imshow(src)
    dst = src.copy()
    gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
    canny = cv2.Canny(gray, 5000, 1500, apertureSize = 5, L2gradient = True)
    #min_theta=0 max_theta=np.pi/2
 
    lines = cv2.HoughLines(canny, 0.8, np.pi / 180, 220, srn = 100, stn = 200, min_theta = 89, max_theta = 91)

    miny=700

    for i in lines:
        rho, theta = i[0][0], i[0][1]
        #print(theta)
        a, b = np.cos(theta), np.sin(theta)
        x0, y0 = a*rho, b*rho
        
        scale = src.shape[0] + src.shape[1]

        x1 = int(x0 + scale * -b)
        y1 = int(y0 + scale * a)
        x2 = int(x0 - scale * -b)
        y2 = int(y0 - scale * a)
        #print("y1",y1,"y2",y2)
        if (y2 < miny):
            miny=y2
        cv2.line(dst, (x1, y1), (x2, y2), (0, 0, 255), 2)
        #cv2.circle(dst, (x0, y0), 3, (255, 0, 0), 1, cv2.FILLED)
        header = src[:miny, :]
        body=src[miny+10:,:]
    #print(miny)
    #cv2_imshow(body)
        
    return header,body
      
     
       



In [ ]:
def cropCenter(img):
    src=img
    dst=src.copy
    w=int (img.shape[1]/2)
    left=src[:,:w-5]
    right=src[:,w+5:]
    print("left")
    cv2_imshow(left)
    print("right")
    cv2_imshow(right)
    contour(left)
    contour(right)


In [ ]:
def cropCenter(img):
    src=img
    dst = src.copy()
    gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
    canny = cv2.Canny(gray, 5000, 1500, apertureSize = 5, L2gradient = True)
    lines = cv2.HoughLines(canny, 0.8, np.pi / 180, 200, srn = 100, stn = 200, min_theta = 0, max_theta = np.pi/8)
    row=0
    col=0
    for i in lines:
        rho, theta = i[0][0], i[0][1]
        a, b = np.cos(theta), np.sin(theta)
        x0, y0 = a*rho, b*rho
        print("x0",x0,"y0",y0)
        scale = src.shape[0] + src.shape[1]

        x1 = int(x0 + scale * -b)
        y1 = int(y0 + scale * a)
        x2 = int(x0 - scale * -b)
        y2 = int(y0 - scale * a)
        row=x2
        col=y2
        cv2.line(dst, (x1, y1), (x2, y2), (0, 0, 255), 2)
    #cv2.circle(dst, (x0, y0), 3, (255, 0, 0), 1, cv2.FILLED)

    left = src[:, :row-15]
    right= src[:,row+15:]
   
    #cv2_imshow(dst)
    #contour(left)
    #contour(right)

In [ ]:

#반페이지를 입력받고 크롭하기 
def contour(page_rl):
    print("contour")

    #이미지 흑백화 
    imgray = cv2.cvtColor(page_rl, cv2.COLOR_BGR2GRAY) 
    img2=imgray.copy()
    #이미지 이진화 (스캔본 처럼)
    blur = cv2.GaussianBlur(imgray, (3,3), 0)
    thresh = cv2.threshold(blur, 70, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]


    # Morph operations
    edge = cv2.Canny(imgray, 100, 200)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(1000,200))
    closed = cv2.morphologyEx(edge, cv2.MORPH_CLOSE, kernel)

	
    contours, hierarchy = cv2.findContours(closed.copy(),cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    

    contours_xy = np.array(contours , dtype=object)
    contours_xy.shape
    #한페이지 내에서 문제 순서대로 불러오기
    contours=reversed(contours)


	#한페이지 내의 모든 폐곡선 범위에 대해 실행 
    top=[]
    for c in contours:

		#폐곡선 바운더리 
        x,y,w,h = cv2.boundingRect(c)
        top.append(y)
        #cv2.rectangle(page_rl, (x,y), (w,h), (0,0,255), 3) 
        #img_trim = page_rl[y:y+h+20 , :]
        #cv2_imshow(img_trim)
            #cv2.imwrite('/content/drive/MyDrive/GRADING_Study/kh/trim/'+str (qnum)+'.png',img_trim)
            #cv2_imshow(page_rl)
            #include(img_trim)
    print(top)
    total=len(top)-1
    global qnum
    for i in range(total):
        qnum+=1
        if (i==0):
            img_trim=page_rl[top[i]:top[i+1]-5,:]
        else:
            img_trim=page_rl[top[i]-10:top[i+1]-5,:]
        #cv2.imwrite('/content/MyDrive/GRADIING_Study/kh/trim/'+str(qnum)+'.png',img_trim)
        print(qnum)
        #cv2_imshow(img_trim)

In [ ]:

def include(cropped):
  
    d = pytesseract.image_to_string(cropped, lang='eng+Hangul')
    d=d.replace(' ','')
    print(d)